In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

import dateutil.parser

In [20]:
# dictionary of text dates and links (March 2017 - June 2017)

# Febuary 12, 2017
url1 = 'https://www.reuters.com/article/us-airlines-united-pilot-idUSKBN15R0RC'

# MARCH 7, 2017
url2 = 'https://www.reuters.com/article/us-united-airlines-idUSKBN16E2P2'

#MARCH 26, 2017
url3 = 'https://www.reuters.com/article/us-unitedairlines-leggings-idUSKBN16X13G'

# April 7, 2017
url4 = 'https://www.reuters.com/article/us-usair-united-idUSTRE6365PN20100408'

# April 11, 2017
url5 = 'https://www.reuters.com/article/us-ual-passenger-shares-idUSKBN17D1L7'

# April 14, 2017
url6 = 'https://www.reuters.com/article/us-ual-scorpion-idUSKBN17G1EQ'

# APRIL 17, 2017
url7 = 'https://www.reuters.com/article/us-ual-passenger-couple-idUSKBN17J05Q'

# April 27, 2017
url8 = 'https://www.reuters.com/article/us-ual-passenger-idUSKBN17T2WM'

# May 2, 2017
url9 = 'https://www.reuters.com/article/us-ual-passenger-ceo-idUSKBN17Y1J8'

#MAY 4, 2017
url10 = 'https://www.reuters.com/article/ual-passenger-idUSL1N1I51WE'


#MAY 30, 2017
url11 = 'https://www.reuters.com/article/us-united-fine-idUSKBN18Q2EQ'

# June 3, 2017
url12 = 'https://www.reuters.com/article/us-venezuela-airlines-united-idUSKBN18U0TL'



url_dict = {'2017-02-12':url1,
            '2017-03-07':url2,
            '2017-03-26':url3,
            '2017-04-07':url4,
            '2017-04-11':url5,
            '2017-04-14':url6,
            '2017-04-17':url7,
            '2017-04-27':url8,
            '2017-05-02':url9,
            '2017-05-04':url10,
            '2017-05-30':url11,
            '2017-06-03':url12,
           }


In [21]:
# extract info from each website and store it in dataframe
df_text = pd.DataFrame(columns = ['Date','Header','Text'])
header = []
text = []
date = []
for key in url_dict:
    page = requests.get(url_dict[key]).text
    soup = BeautifulSoup(page, 'html.parser')
    date.append(datetime.strptime(key, '%Y-%m-%d').date())
    url_header = soup.find('h1').text
    header.append(url_header)
    url_text = soup.find('div', class_ ='ArticleBodyWrapper')
    url_text = url_text.text
    text.append(url_text)
df_text['Date']=date
df_text['Header']=header
df_text['Text']=text

# remove the author and "min read" sections
df_text_temp = [] 
for text in df_text['Text']:
    loc = text.find('(Reuters)')
    df_text_temp.append(text[loc+9:])
df_text['Text'] = df_text_temp




In [22]:
df_text['Date'] = df_text['Date'].apply(lambda x: pd.to_datetime(x))
df_text.dtypes

Date      datetime64[ns]
Header            object
Text              object
dtype: object

In [23]:
# import stock data of netflix
df_stock = pd.read_csv('UAL.csv', header=0)

df_stock['Date']=df_stock['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


df_stock = df_stock.drop(['High','Low','Close','Adj Close'], axis=1)
df_stock.dtypes

Date      datetime64[ns]
Open             float64
Volume             int64
dtype: object

In [24]:
# Merge two dataframes
df = pd.merge(df_text, df_stock, how='inner', on='Date')
df = df.sort_values('Date')

In [25]:
# show % change in stock price after news article publish
pct_stock = df['Open'].pct_change()*100
df['% Change in Stock Price'] = pct_stock
pct_volume = df['Volume'].pct_change()*100
df['% Change in Volume'] = pct_volume
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316


# Processing the text


## import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [26]:
# import spacy libraries
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_sm')

In [27]:

text_nlp = nlp(df['Text'][1]) 
# Remove stop words
text_nlp_clean = [word for word in text_nlp if word.is_stop == False]
# Lemmatize words
text_nlp_clean = [word.lemma_ for word in text_nlp_clean]

In [28]:
text_clean = []
for ind, row in df.iterrows():
    
    # lower casing
    text_lower = row['Text'].lower()
    
    # tokenization
    text_nlp = nlp(row['Text']) 
    
    # remove punctuation and empty space
    text_nlp_clean = [word for word in text_nlp if not word.is_punct | word.is_space]
    
    # Remove stop words
    text_nlp_clean = [word for word in text_nlp_clean if word.is_stop == False]
    
    # Lemmatize tokens
    text_nlp_clean = [word.lemma_ for word in text_nlp_clean]
    
    # add preprocessed text to list
    text_clean.append(text_nlp_clean)

# turn list of words to string
def to_string(list1):
    sentence = ""
    
    
    # traverse in the string   
    for word in list1:  
        sentence += str(word) 
    
    # return string   
    return sentence

# turn list of strings to string
to_string = []
for text in text_clean:
    to_string.append(' '.join(text))


df['Text (clean)'] = to_string
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean)
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN,United Continental Holdings UAKL.N interested ...
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097,United Airlines pilot union lash effort merge ...
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562,United Airlines UAL.N chief executive face mou...
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289,engage couple remove United Airlines flight Co...
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316,United Airlines UAL.N passenger drag Chicago f...


# Using a Pretrained Model 

In [29]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [30]:
# a function to return sentiment score
def polarity_score(text):
    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(text)['compound']

vader_score = []
for text in df['Text (clean)']:
    score = polarity_score(text)
    vader_score.append(score)

df['Vader Sentiment Score'] = vader_score

In [31]:
df.head()

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN,United Continental Holdings UAKL.N interested ...,0.9485
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097,United Airlines pilot union lash effort merge ...,0.9981
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562,United Airlines UAL.N chief executive face mou...,0.9877
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289,engage couple remove United Airlines flight Co...,0.9501
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316,United Airlines UAL.N passenger drag Chicago f...,0.9136


## Sentiment Analysis

In [32]:
# run this cell to import nltk
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 
from utils import process_tweet, build_freqs

In [33]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aashi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:

# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [35]:
# split the data into two pieces (80-20), one for training and one for testing (validation set)  
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [36]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [37]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [38]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11340


In [39]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        
        x[0,1] += freqs.get((word,1),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word,0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [40]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

In [41]:
Y.shape

(8000, 1)

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import Pipeline

### SVM Parameters

In [43]:
param_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1]}  

In [44]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, Y)

C:\Users\aashi\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(random_state=0)

In [45]:

# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [46]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet_prob(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = classifier.predict_proba(x)
    
    ### END CODE HERE ###
    
    return y_pred

In [47]:

my_tweet = 'The plot was terrible and I was sad until the ending!'
predict_tweet(my_tweet, freqs)

array([0.])

# Testing the Model 

In [48]:
sentiment = []
sentiment_prob = []
for i in df.iloc[:,2]:
    sentiment.append(predict_tweet(i, freqs)[0])
    sentiment_prob.append(predict_tweet_prob(i, freqs)[0][1])

In [49]:
sentiment


[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [50]:
sentiment_prob

[0.6938569763837213,
 0.000992856711037133,
 0.016640031758956002,
 0.10050501782597547,
 0.003088541074744996,
 5.593078354866826e-05,
 0.03852008349259754,
 0.5557759528050944]

In [51]:
df['sentiment'] = sentiment
df['sentiment_prob'] = sentiment_prob
df

,Date,Header,Text,Open,Volume,% Change in Stock Price,% Change in Volume,Text (clean),Vader Sentiment Score,sentiment,sentiment_prob
0,2017-03-07,"United looking at second-hand aircraft, rules ...",- United Continental Holdings UAKL.N is inter...,72.690002,3764000,NaN,NaN,United Continental Holdings UAKL.N interested ...,0.9485,1.0,0.693857
1,2017-04-07,UAL pilots attack U.S. Airways merger prospect,- United Airlines' pilots union lashed out ag...,69.970001,2471100,-3.741919,-34.349097,United Airlines pilot union lash effort merge ...,0.9981,0.0,0.000993
2,2017-04-11,United Airlines faces mounting pressure over h...,- United Airlines UAL.N and its chief executi...,70.150002,17696500,0.257255,616.138562,United Airlines UAL.N chief executive face mou...,0.9877,0.0,0.016640
3,2017-04-17,United Air removes couple traveling to wedding...,- An engaged couple were removed from a Unite...,69.510002,4761900,-0.912331,-73.091289,engage couple remove United Airlines flight Co...,0.9501,0.0,0.100505
4,2017-04-27,United Airlines reaches settlement with passen...,- United Airlines UAL.N and the passenger who...,69.769997,5512200,0.374040,15.756316,United Airlines UAL.N passenger drag Chicago f...,0.9136,0.0,0.003089
5,2017-05-02,U.S. lawmakers grill airline executives after ...,- U.S. lawmakers threatened United Airlines U...,70.430000,5434700,0.945970,-1.405972,U.S. lawmaker threaten United Airlines UAL.N c...,0.9460,0.0,0.000056
6,2017-05-04,United Air to face second congressional grilli...,- United Airlines will be back in the hot sea...,74.559998,2886300,5.863976,-46.891273,United Airlines hot seat Thursday U.S. Congres...,0.9921,0.0,0.038520
7,2017-05-30,"U.S. proposes fining United $435,000 over 2014...",- The U.S. Federal Aviation Administration on...,80.900002,3428700,8.503224,18.792225,U.S. Federal Aviation Administration Tuesday p...,0.8625,1.0,0.555776
